In [ ]:
from SynTemp.SynAAM.atom_mappers import map_with_local_mapper, map_with_rdt
from SynTemp.SynAAM.consensus_aam import ConsensusAAM

In [ ]:
from SynTemp.SynAAM.run_consensus import run_consensus_aam
from SynTemp.utils import load_database
from rxnmapper import RXNMapper
rxn_mapper = RXNMapper()
data = load_database('./Data/jaworski/jaworski_reactions.json.gz')[:5]

In [ ]:
mapper_types = ['graphormer', 'local_mapper']
mapped_reactions = run_consensus_aam(data, check_balance=False, mapper_types=mapper_types, rdt_jar_path='../Data/RDT_2.4.1.jar', working_dir='./',
                            data_name='Jaworski_8', n_jobs=1)

In [ ]:
import sys
sys.path.append('../')
from SynTemp.SynAAM.aam_postprocess import AMMPostprocessor
from SynTemp.SynITS.its_extraction import ITSExtraction
from SynTemp.utils import load_database, save_database

data = load_database(f'./Data/uspto//uspto_balance_aam_reactions.json.gz')
mapper_name = ['rxn_mapper', 'graphormer', 'local_mapper']

# make sure reactions all have atom mapping
check_valid = AMMPostprocessor.parallel_postprocess(data, mapper_name, threshold=3, n_jobs=5, verbose=1)
valid_data = [reaction for reaction in check_valid if reaction.get('Valid')]

In [ ]:
golden = pd.read_csv('./Data/golden/golden_dataset.csv')
golden['reactions']

In [ ]:
data_check2 = load_database('./Data/golden/golden_aam_reactions.json.gz')
data_check2[0]

In [ ]:
import pandas as pd
data_check =pd.DataFrame(data_check2)
data_check['ground_truth'] = golden['reactions']

In [ ]:
import pandas as pd
data_check = pd.read_csv('./Data/benchmark.csv')
data_check

In [25]:
data_check[0]

{'R-id': 'G_0',
 'reactions': 'C=CCC(N)(c1ccccc1)c1ccccc1.O=CC(C(=O)N1CCOCC1)c1ccccc1>>C=CCC(N=C(c1ccccc1)c1ccccc1)C(C(=O)N1CCOCC1)c1ccccc1',
 'local_mapper': '[CH2:1]=[CH:2][CH2:3][C:6]([NH2:5])([c:7]1[cH:8][cH:9][cH:10][cH:11][cH:12]1)[c:13]1[cH:14][cH:15][cH:16][cH:17][cH:18]1.O=[CH:4][CH:19]([C:20](=[O:21])[N:22]1[CH2:23][CH2:24][O:25][CH2:26][CH2:27]1)[c:28]1[cH:29][cH:30][cH:31][cH:32][cH:33]1>>[CH2:1]=[CH:2][CH2:3][CH:4]([N:5]=[C:6]([c:7]1[cH:8][cH:9][cH:10][cH:11][cH:12]1)[c:13]1[cH:14][cH:15][cH:16][cH:17][cH:18]1)[CH:19]([C:20](=[O:21])[N:22]1[CH2:23][CH2:24][O:25][CH2:26][CH2:27]1)[c:28]1[cH:29][cH:30][cH:31][cH:32][cH:33]1',
 'rxn_mapper': '[CH2:1]=[CH:2][CH2:3][C:6]([NH2:5])([c:7]1[cH:8][cH:9][cH:10][cH:11][cH:12]1)[c:13]1[cH:14][cH:15][cH:16][cH:17][cH:18]1.O=[CH:4][CH:19]([C:20](=[O:21])[N:22]1[CH2:23][CH2:24][O:25][CH2:26][CH2:27]1)[c:28]1[cH:29][cH:30][cH:31][cH:32][cH:33]1>>[CH2:1]=[CH:2][CH2:3][CH:4]([N:5]=[C:6]([c:7]1[cH:8][cH:9][cH:10][cH:11][cH:12]1)[c:13]1[cH:14]

In [26]:
data_check[1]

{'R-id': 'G_1',
 'reactions': 'C=CCSCC(C)=O.C=CC(O)C(C)N>>CC(=O)CSCC=CC(O)C(C)N',
 'local_mapper': 'C=[CH:7][CH2:6][S:5][CH2:4][C:2]([CH3:1])=[O:3].C=[CH:8][CH:9]([OH:10])[CH:11]([CH3:12])[NH2:13]>>[CH3:1][C:2](=[O:3])[CH2:4][S:5][CH2:6][CH:7]=[CH:8][CH:9]([OH:10])[CH:11]([CH3:12])[NH2:13]',
 'rxn_mapper': 'C=[CH:7][CH2:6][S:5][CH2:4][C:2]([CH3:1])=[O:3].C=[CH:8][CH:9]([OH:10])[CH:11]([CH3:12])[NH2:13]>>[CH3:1][C:2](=[O:3])[CH2:4][S:5][CH2:6][CH:7]=[CH:8][CH:9]([OH:10])[CH:11]([CH3:12])[NH2:13]',
 'graphormer': '[CH3:7][C:6](=[O:8])[CH2:5][S:4][CH2:3][CH:2]=[CH2:1].[NH2:15][CH:13]([CH3:14])[CH:11]([OH:12])[CH:10]=[CH2:9]>>[CH:10](=[CH:2][CH2:3][S:4][CH2:5][C:6]([CH3:7])=[O:8])[CH:11]([OH:12])[CH:13]([NH2:15])[CH3:14]',
 'ground_truth': '[CH3:1][C:2](=[O:3])[CH2:4][S:5][CH2:6][CH:7]=[CH2:8].[NH2:11][CH:10]([CH3:9])[CH:12]([OH:13])[CH:14]=[CH2:15]>>[CH:14](=[CH:7][CH2:6][S:5][CH2:4][C:2]([CH3:1])=[O:3])[CH:12]([OH:13])[CH:10]([NH2:11])[CH3:9]'}

In [19]:
data_check[4]

{'R-id': 'G_4',
 'reactions': 'CN(CC#Cc1ccsc1)c1ccccc1.O>>CN(C(=CC=O)c1ccsc1)c1ccccc1',
 'local_mapper': '[CH3:1][N:2]([CH2:5][C:4]#[C:3][c:7]1[cH:8][cH:9][s:10][cH:11]1)[c:12]1[cH:17][cH:16][cH:15][cH:14][cH:13]1.[OH2:6]>>[CH3:1][N:2]([C:3](=[CH:4][CH:5]=[O:6])[c:7]1[cH:8][cH:9][s:10][cH:11]1)[c:12]1[cH:13][cH:14][cH:15][cH:16][cH:17]1',
 'rxn_mapper': '[CH3:1][N:2]([CH2:3][C:4]#[C:5][c:7]1[cH:8][cH:9][s:10][cH:11]1)[c:12]1[cH:13][cH:14][cH:15][cH:16][cH:17]1.[OH2:6]>>[CH3:1][N:2]([C:3](=[CH:4][CH:5]=[O:6])[c:7]1[cH:8][cH:9][s:10][cH:11]1)[c:12]1[cH:13][cH:14][cH:15][cH:16][cH:17]1',
 'graphormer': '[OH2:17].[cH:12]1[cH:13][cH:14][cH:15][cH:16][c:11]1[N:2]([CH3:1])[CH2:3][C:4]#[C:5][c:6]1[cH:10][s:9][cH:8][cH:7]1>>[cH:12]1[cH:13][cH:14][cH:15][cH:16][c:11]1[N:2]([CH3:1])[C:5](=[CH:4][CH:3]=[O:17])[c:6]1[cH:10][s:9][cH:8][cH:7]1',
 'ground_truth': '[CH:8]=1[S:9][CH:10]=[C:6]([C:5]#[C:4][CH2:3][N:2]([C:11]2=[CH:12][CH:13]=[CH:14][CH:15]=[CH:16]2)[CH3:1])[CH:7]=1.[OH2:17]>>[C:5]([N:2]([C

In [ ]:
from SynTemp.SynAAM.aam_validator import AMMValidator
mapped_cols = ['rxn_mapper', 'graphormer', 'local_mapper']
validation_results = AMMValidator.validate_smiles(data_check, 'ground_truth', mapped_cols, check_method='RC', ignore_aromaticity=False, n_jobs=4, verbose=2)
validation_results

In [23]:
validation_results[1]['accuracy']

0.8924905456509995

In [18]:
pd.DataFrame(validation_results)

,mapper,accuracy,results
0,rxn_mapper,0.835764,"[False, True, True, True, False, False, False,..."
1,graphormer,0.892491,"[False, True, True, True, True, False, False, ..."
2,local_mapper,0.852512,"[False, True, True, True, True, False, False, ..."


In [ ]:
from SynTemp.SynAAM.aam_validator import AMMValidator

accuracy, results = AMMValidator.validate_smiles(data_check, 'ground_truth', 'graphormer', check_method='RC', ignore_aromaticity=False, n_jobs=2, verbose=2)
accuracy

In [ ]:
from SynTemp.SynAAM.aam_validator import AMMValidator

accuracy, results = AMMValidator.validate_smiles(data_check, 'ground_truth', 'local_mapper', check_method='RC', ignore_aromaticity=False, n_jobs=2, verbose=2)
accuracy

In [ ]:
accuracy

In [ ]:
from SynTemp.SynAAM.aam_validator import AMMValidator

accuracy, results = AMMValidator.validate_smiles(data_check, 'ground_truth', 'mapper', check_method='RC', ignore_aromaticity=False, n_jobs=1, verbose=2)

In [ ]:
accuracy

In [ ]:
test = valid_data[:]
test[0]

In [ ]:
AMMValidator.validate_smiles(test, 'local_mapper', 'graphormer', check_method='RC', ignore_aromaticity=False, n_jobs=1, verbose=2)

In [ ]:
mapped_smile = valid_data[0]['rxn_mapper']
ground_truth = valid_data[0]['local_mapper']
checker = AMMValidator.smiles_check(mapped_smile, ground_truth, check_method='RC', ignore_aromaticity=False)

In [ ]:
AMMValidator.smiles_check(mapped_smile, ground_truth, check_method='ITS', ignore_aromaticity=False)

In [ ]:
AMMValidator.smiles_check(mapped_smile, ground_truth, check_method='RC', ignore_aromaticity=False)

In [ ]:
import pandas as pd
data_check = pd.read_csv('./Data/benchmark.csv', index_col=0)
data_check = data_check.to_dict('records')

In [ ]:
from SynTemp.SynITS.its_extraction import ITSExtraction
#mapper_name = ['rxn_mapper', 'graphormer', 'local_mapper', 'ground_truth']
mapper_name = ['ground_truth', 'rxn_mapper', 'graphormer', 'local_mapper']
# Extract rules and graph rules based on Imaginary Transition State (ITS)
its_graph, its_graph_wrong = ITSExtraction.parallel_process_smiles(data_check, mapper_name, threshold=3, n_jobs=4, verbose=1, export_full=False, check_method='RC')

len(its_graph)

In [ ]:
len(its_graph_wrong)

In [ ]:
from SynTemp.SynITS.graph_rules_extraction import GraphRuleExtraction
key = 0
ground_its = its_graph_wrong[key]['ground_truth']
ground_rules =GraphRuleExtraction.extract_reaction_rules(ground_its[0], ground_its[1], ground_its[2], extend=False)

rxn_its = its_graph_wrong[key]['rxn_mapper']
rxn_rules =GraphRuleExtraction.extract_reaction_rules(rxn_its[0], rxn_its[1], rxn_its[2], extend=False)

graph_its = its_graph_wrong[key]['graphormer']
graph_rules = GraphRuleExtraction.extract_reaction_rules(graph_its[0], graph_its[1], graph_its[2], extend=False)

local_its = its_graph_wrong[key]['local_mapper']
local_rules = GraphRuleExtraction.extract_reaction_rules(local_its[0], local_its[1], local_its[2], extend=False)

In [ ]:
for i in ground_rules[2].nodes(data=True):
    print(i)

In [ ]:
for i in rxn_rules[2].nodes(data=True):
    print(i)

In [ ]:
for i in graph_rules[2].nodes(data=True):
    print(i)


In [ ]:
for i in local_rules[2].nodes(data=True):
    print(i)

In [ ]:
from SynTemp.SynVis.chemical_graph_vis import ChemicalGraphVisualizer
vis = ChemicalGraphVisualizer(seed = 42)
vis.visualize_all(ground_rules)

In [ ]:
from SynTemp.SynVis.chemical_graph_vis import ChemicalGraphVisualizer
vis = ChemicalGraphVisualizer(seed =42)
vis.visualize_all(rxn_rules)

In [ ]:
vis.visualize_all(graph_rules)

In [ ]:
vis.visualize_all(local_rules)

In [ ]:
vis.visualize_all(its_graph_wrong[0]['graphormer'])

In [ ]:
vis.visualize_all(its_graph_wrong[0]['local_mapper'])

In [ ]:
347/1855